# Постановка задачи и мотивация

Необходимо разработать инструмент для группировки идентичных наименований товаров

Обычно такие инструменты используют для поддержания базы товаров в надлежащем виде:
много товаров при постановке на учет имеют немного измененные названия (например в зависимости от поставщика), а для точного учета дубликаты в базе недопустимы

Как правило у клиентов уже есть первичная база с эталонными названиями для существующих продуктов, если же их нет, то первым делом нужно собрать подобный список названий.

Далее при поступлении новых товаров, производится поиск в базе эталонов и в полу-автоматическом режиме выбирается соответствует ли товар какому-то из эталонов, или это новый уникальный товар и должен пополнить список эталонных названий

В нашем случае списка эталонных названий нет, т.е. задачу необходимо разбить на две логических части:
<ul>
<li>составление списка эталонных названий</li>
<li>разработка инструмента для поиска совпадений с заданным списком эталонов</li>
</ul>

как правило данные списки довольно большие, поэтому будем пытаться разработать алгоритм работающий батчами и параллельно

обычно в названиях бывают различные сокращения / опечатки , поэтому нужно строить алгоритм устойчивый к такого рода вещам

### поиск совпадения с эталонами
т.к. в списке эталонов не должно быть дубликатов, нет необходимости реализовывать алгоритмы кластеризации, достаточно реализовать поиск k-ближайших соседей с отсечением по порогу по метрике близости.

Дополнительно стоит отметить, что в такой базе разные производители, граммовки, страна производства, год сбора урожая (для вина) и т.п. являются критическими факторами, но при этом они не всегда указываются поставщиком товаров в магазин


### составление списка эталонов
Для упрощения составления списка эталонных названий может использоваться тот же алгоритм, который предназначен для поиска совпадений с эталонами

В любом случае данная процедура должна иметь человеческий контроль

Для лучшего качества, необходимо итеративно провести процесс, до тех пор, пока не будет дальнейших изменений списка эталонов


In [1]:
import json
from itertools import chain
from transliterate import translit, exceptions
import re
import nltk
import pandas as pd
from functools import lru_cache
from sklearn.base import TransformerMixin, ClassifierMixin
import sparse_dot_topn.sparse_dot_topn as ct
from scipy.sparse import csr_matrix, vstack
import numpy as np
from multiprocessing import Pool, cpu_count
import pymorphy2
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
import os
import gc
import warnings
from collections import Counter

warnings.simplefilter('ignore')

/home/dolgop/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
with open('test_task_NLP.json') as f:
    texts_clustered = json.load(f)

In [3]:
texts = list(chain(*texts_clustered))

# Разведочный анализ

In [4]:
len(texts), len(texts_clustered)

(3580, 1223)

#####  страна производитель иногда пишется, а иногда нет
стоит экстрактировать страну, в отдельный признак (и собирать данные соответствующим способом в будущем)

In [5]:
texts[3], texts[5]

('Блинчики с мясом, Уже Готово , 140 г',
 'Блинчики с мясом, Уже Готово , 220 г, Россия')

видна так же проблема с одним и тем же товаром в разных объемах/ разных годов и тп, т.е. стоит обрабатывать цифры в особом порядке

#### названия иногда на английском языке, а иногда транслитом на русском
стоит все перевести на русский язык

аналогично производитель иногда пишется иногда нет, стоит выносить его в отдельный признак, и собирать новые данные уже с учетом этого

In [6]:
texts[-1000], texts[-999]

('Пицца Ристоранте специальная , Dr.Oetker, 330 г',
 'Пицца Dr.Oetker Ristorante Специале 330г')

# перейдем к разработке алгоритма

In [7]:
# дополнительные разделители для номеров и размерностей
XN_LIST = [u'X', u'x', u'Х', u'х', u'н', u'N', u'n', u'H']
STOP_LIST = ['-', 'X', 'x', 'Х', 'х', 'N', '.', ',', ';', '"', "'", ':', '/',
             '\\', '(', ')', '[', ']', '{', '}', '~', '_', '!', '...', '!!', '!!!']

MORPHER = pymorphy2.MorphAnalyzer()
CPU = cpu_count()

In [8]:
def ngrams(string, n=3):
    '''
    функция приводит строку к нижнему регистру и
    разбивает её на символьные n-граммы

    string - строка (название лекарства после предобработки)
    n - число букв в n-грамме
    '''
    string = string.lower()
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

def batcher(df, size):
    return (df.iloc[pos:pos + size] for pos in range(0, df.shape[0], size))

def numbers_closeness(numbers_1, numbers_2, lam=2):
    numbers_1 = numbers_1.split(';')
    numbers_2 = numbers_2.split(';')
    counts_1 = Counter(numbers_1)
    counts_2 = Counter(numbers_2)
    common = set(numbers_1) & set(numbers_2)
    inter = 0
    for number in common:
        inter += min(counts_1[number], counts_2[number])
    uni = len(numbers_1) + len(numbers_2) - inter
    return ((inter + lam) * 1. / (uni + lam))


def extract_numbers(text):
    numbers = []
    for token in text.split():
        try:
            float(token.replace(',','.'))
            token = token.replace(',','.')
            numbers.append(token)
        except ValueError:
            continue
    return ';'.join(numbers)

def top_k(match_sparse, row_number, k=3):
    row = match_sparse[row_number]
    ix = row.data.argsort()[-k:][::-1]
    top_k_indicies = row.indices[ix]
    top_k_values = row.data[ix]
    return top_k_indicies, top_k_values


def top_n_idx_sparse(matrix, n):
    result_list = []
    count = 0
    for le, ri in (zip(matrix.indptr[:-1], matrix.indptr[1:])):
        n_row_pick = min(n, ri - le)
        ix = le + np.argsort(matrix.data[le:ri])[-n_row_pick:]
        result_list.append(
            np.concatenate([[matrix.indices[ix]],
                            [matrix.data[ix]],
                            [np.arange(n_row_pick)],
                            [np.ones(shape=(n_row_pick,))*count]
                           ], axis=0).T
        )
        count += 1
    return np.concatenate(result_list)


@lru_cache(maxsize=None)
def parse_token(token):
    return MORPHER.parse(token)[0].normal_form


class NameCleanTransformer(TransformerMixin):
    def __init__(self, njobs=CPU):
        self.njobs=njobs

    @staticmethod
    def do_morphy(text):
        new_text = []
        for token in nltk.word_tokenize(text):
            new_text.append(parse_token(token))
        return ' '.join(new_text)

    @staticmethod
    def translit_text(x, lang='ru'):
        tokens = nltk.word_tokenize(x)
        final_text = ""
        for tok in tokens:
            try:
                tr = translit(tok, lang)
                final_text += tr + u' '
            except exceptions.LanguageDetectionError:
                final_text += tok + u' '
        return final_text

    @staticmethod
    def nan_to_empty(x):
        if (pd.isnull(x)) | (x == "-") | (x == "_"):
            return ''
        else:
            return x.lower()

    @staticmethod
    def preprocess_name(names, split_rule=r'(\d+[,|.]?\d+|\d+|\/|\+|!|№|\.|,|\(|\)|%|\*|~|"|_)'):
        '''
        функия предобработки названий
        на выходе - отсортированный лексикографически лист из токенов

        names - названия
        '''
        ext_name_clean = []
        for string in names:
            res = []
            for part in string.split():
                results = re.split(split_rule, part)
                results = list(filter(None, results))
                results = NameCleanTransformer.split_dashes(results)
                res.extend(results)
            res = NameCleanTransformer.split_NX(res)
            res = [w for w in res if w not in STOP_LIST]
            res = sorted(res)
            ext_name_clean.append(' '.join(res))
        return pd.Series(ext_name_clean)

    @staticmethod
    def split_dashes(results):
        '''
        функция обработки дефисов
        если дефис в середине, то не разрываем название
        если дефис по краям, отделяем его

        results - list, части названия
        '''
        results_clean = []
        for r in results:
            l_dash = False
            if len(r) > 1:
                if r[0] == '-':
                    results_clean.append('-')
                    r = r[1:]
                if r[-1] == '-':
                    r = r[:-1]
                    l_dash = True
                results_clean.append(r)
                if l_dash:
                    results_clean.append('-')
            else:
                results_clean.append(r)
        return results_clean

    @staticmethod
    def split_NX(results, rule=r'(Х|х|X|x|N|n|Н)'):
        '''
        функция разбиения названия
        цифры отделяются от слов, применяются разделители

        results - list, части названия
        '''
        results_clean = []
        if len(results) == 1:
            return results
        for idx in range(len(results)):
            r = results[idx]
            if not len(set(XN_LIST) & set(r)) > 0:
                results_clean.append(r)

            else:
                if idx == 0:
                    if any(i.isdigit() for i in results[idx + 1]) and r[-1] in rule:
                        r = re.split(rule, r)
                        r = list(filter(None, r))

                elif idx == len(results) - 1:
                    if any(i.isdigit() for i in results[idx - 1]) and r[0] in rule:
                        r = re.split(rule, r)
                        r = list(filter(None, r))

                else:
                    if any(i.isdigit() for i in results[idx + 1]) and r[-1] in rule or \
                                    any(i.isdigit() for i in results[idx - 1]) and r[0] in rule:
                        r = re.split(rule, r)
                        r = list(filter(None, r))

                if isinstance(r, list):
                    results_clean.extend(r)
                else:
                    results_clean.append(r)
        return results_clean

    def fit(self, X, y=None, **fit_params):
        pass

    @staticmethod
    def make_transform(X):
        X_new = NameCleanTransformer.preprocess_name(X)
        X_new = X_new.apply(lambda x: NameCleanTransformer.translit_text(x.lower()))
        X_new = X_new.apply(NameCleanTransformer.do_morphy)
        return X_new

    def fit_transform(self, X, y=None, **fit_params):
        if self.njobs > 1:
            data_split = np.array_split(X, self.njobs)
            pool = Pool(self.njobs)
            X_new = pd.concat(pool.map(NameCleanTransformer.make_transform, data_split))
            pool.close()
            pool.join()
        else:
            X_new = NameCleanTransformer.make_transform(np.array(X))
        return X_new

    def transform(self, X, y=None, **fit_params):
        if self.njobs > 1:
            data_split = np.array_split(X, self.njobs)
            pool = Pool(self.njobs)
            X_new = pd.concat(pool.map(NameCleanTransformer.make_transform, data_split))
            pool.close()
            pool.join()
        else:
            X_new = NameCleanTransformer.make_transform(np.array(X))
        return X_new


class CandidatesSelector(ClassifierMixin):
    def __init__(self, n_jobs=CPU, n_top=1000, lower_bound=0.1, batch_size=50000):
        self.n_top = n_top
        self.n_jobs = n_jobs
        self.lower_bound = lower_bound
        self.batch_size = batch_size
        pass

    def fit(self, B, y=None, **fit_params):
        self.B = B.T.tocsr()

    def find_top_n(self, A):
        M, _ = A.shape
        idx_dtype = np.int32
        indptr = np.zeros(M + 1, dtype=idx_dtype)
        nnz_max = M * self.n_top
        indices = np.zeros(nnz_max, dtype=idx_dtype)
        data = np.zeros(nnz_max, dtype=A.dtype)
        _, N = self.B.shape

        ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(self.B.indptr, dtype=idx_dtype),
            np.asarray(self.B.indices, dtype=idx_dtype),
            self.B.data,
            self.n_top,
            self.lower_bound,
            indptr, indices, data)
        return csr_matrix((data, indices, indptr), shape=(M, N))

    def predict(self, A):
        A = A.tocsr()
        ixs = [int(i) for i in np.linspace(0, A.shape[0], int(np.ceil(A.shape[0] / self.batch_size)+1))]
        iter_num = 0
        results = []
        while iter_num*self.n_jobs < len(ixs)-1:
            A_parts = [A[ixs[i]:ixs[i+1]] for i in range(
                                                    iter_num*self.n_jobs,
                                                    min(iter_num*self.n_jobs+self.n_jobs,
                                                    len(ixs)-1)
                                                    )]
            if self.n_jobs > 1:
                p = Pool(self.n_jobs)
                result = (p.map(self.find_top_n, A_parts))
                p.close()
                p.join()
            else:
                result = [self.find_top_n(A_i) for A_i in A_parts]
            result = vstack(result)
            results.append(result)
            iter_num += 1
        return vstack(results)


In [9]:
def make_final_dataframe(match_sparse, etalons, candidates, k):
    results = top_n_idx_sparse(match_sparse, n=k)
    d = {'CANDIDATE_NAME':               candidates.iloc[results[:, 3]]['CANDIDATE_NAME'].values,
         'ETALON_NAME':                  etalons.iloc[results[:, 0]]['ETALON_NAME'].values,
         'SIMILARITY_SCORE':             results[:, 1],
         'RANK':                         results[:, 2],
         'ETALON_ID':                    etalons.iloc[results[:, 0]]['ETALON_ID'].values,
         'CANDIDATE_ID':                 candidates.iloc[results[:, 3]]['CANDIDATE_ID'].values}
    final_top = pd.DataFrame(d)
    cand_digits = NameCleanTransformer.preprocess_name(final_top['CANDIDATE_NAME']).apply(extract_numbers)
    etalon_digits = NameCleanTransformer.preprocess_name(final_top['ETALON_NAME']).apply(extract_numbers)
    final_top['SIMILARITY_SCORE'] = final_top['SIMILARITY_SCORE'] * np.array([numbers_closeness(candidate_numbers, etalon_numbers) for candidate_numbers, etalon_numbers in zip(cand_digits, etalon_digits)])
    final_top.sort_values(by=['CANDIDATE_ID', 'SIMILARITY_SCORE'], ascending=[True, False], inplace=True)
    final_top.reset_index(drop=True, inplace=True)
    final_top['RANK'] = np.concatenate(final_top.groupby(['CANDIDATE_ID']).SIMILARITY_SCORE.apply(lambda x: x.values.argsort()[::-1]+1).values)
    return final_top

# затестим что получилось

In [10]:
BATCH_SIZE = 100
NUM_NEIGHBOURS = 10

In [11]:
etalons = pd.DataFrame()
etalons['ETALON_NAME'] = texts
etalons['ETALON_ID'] = np.arange(len(etalons))

candidates = pd.DataFrame()
candidates['CANDIDATE_NAME'] = texts
candidates['CANDIDATE_ID'] = np.arange(len(candidates))

In [12]:
names_selector = Pipeline([
    ('transformer', NameCleanTransformer(
                              njobs=CPU)),
    ('tfidf', TfidfVectorizer(analyzer=ngrams)),
    ('CandidatesSelector', CandidatesSelector(
                              n_jobs=CPU,
                              batch_size=BATCH_SIZE)
                            )])
t = time.time()
names_selector.fit(etalons['ETALON_NAME'])
print(time.time() - t)

0.5572464466094971


In [13]:
t = time.time()
match_sparse = names_selector.predict(candidates['CANDIDATE_NAME'])
print('names selector finished', time.time() - t)
t = time.time()


names selector finished 1.0653631687164307


In [14]:
final_top_full = make_final_dataframe(match_sparse, etalons, candidates, NUM_NEIGHBOURS)

In [15]:
# удалим сравнения эталона с самим собой
final_top_full = final_top_full[final_top_full['ETALON_ID']!=
              final_top_full['CANDIDATE_ID']].sort_values(by='SIMILARITY_SCORE', ascending=False)

In [16]:
final_top_full[final_top_full['SIMILARITY_SCORE']>0.9]

,CANDIDATE_NAME,ETALON_NAME,SIMILARITY_SCORE,RANK,ETALON_ID,CANDIDATE_ID
8023,Биойогурт Данон Активиа питьевой чернослив с б...,Биойогурт Данон Активиа питьевой чернослив с б...,1.000000,2,800,803
7994,Биойогурт Данон Активиа питьевой чернослив с б...,Биойогурт Данон Активиа питьевой чернослив с б...,1.000000,1,803,800
8803,Биопродукт Данон Активиа кефирный c бифидобакт...,Биопродукт Данон Активиа кефирный c бифидобакт...,1.000000,2,882,881
31451,Сыр Galbani Моцарелла Мини из коров мол 45% жи...,Сыр Galbani Моцарелла Мини из коров мол 45% жи...,1.000000,2,3146,3147
31641,Сыр Ламбер из коровьего молока 50% жирн 230г Р...,Сыр Ламбер из коровьего молока 50% жирн 230г Р...,1.000000,2,3168,3166
...,...,...,...,...,...,...
4124,Зубная паста Colgate Бережное отбеливание 100 мл,Зубная паста Colgateт бережное отбеливание 100мл,0.900053,5,410,412
4094,"Зубная паста Бережное отбеливание Colgate, 100 мл",Зубная паста Colgateт бережное отбеливание 100мл,0.900053,5,410,409
4144,"Зубная паста Colgate Бережное отбеливание, 100мл",Зубная паста Colgateт бережное отбеливание 100мл,0.900053,5,410,414
29332,Сок Fleur Alpine яблоко осветленное с 4 мес ор...,Сок Fleur Alpine груша осветленная с 4 мес орг...,0.900020,2,2933,2935


In [17]:
final_top_full.to_csv('final_top.csv', index=False)

# приведем вывод к виду, требуемому в задании

возьмем порог похожести для объединения в кластеры, но такой аутпут не самый информативный с точки зрения работы алгоритма, для изучения алгоритма лучше смотреть на final_top_full

In [18]:
TRESHOLD  = 0.8

In [19]:
def merge(lsts):
    sets = [set(lst) for lst in lsts if lst]
    merged = True
    while merged:
        merged = False
        results = []
        while sets:
            common, rest = sets[0], sets[1:]
            sets = []
            for x in rest:
                if x.isdisjoint(common):
                    sets.append(x)
                else:
                    merged = True
                    common |= x
            results.append(common)
        sets = results
    return sets

In [20]:
edges = final_top_full.loc[final_top_full['SIMILARITY_SCORE']>TRESHOLD, ['ETALON_ID', 'CANDIDATE_ID']].values.tolist()

In [21]:
with_clusters = set(final_top_full[final_top_full['SIMILARITY_SCORE']>TRESHOLD]['ETALON_ID']).union(
set(final_top_full[final_top_full['SIMILARITY_SCORE']>TRESHOLD]['CANDIDATE_ID']))

In [22]:
no_clusters = candidates[~candidates['CANDIDATE_ID'].isin(with_clusters)][['CANDIDATE_ID']].values.tolist()

In [23]:
clusters = merge(edges)+no_clusters

In [24]:
clusters_texts = []
for i in clusters:
    clusters_texts.append(candidates[candidates['CANDIDATE_ID'].isin(i)]['CANDIDATE_NAME'].values.tolist())

In [25]:
clusters_texts = sorted(clusters_texts, key=lambda x: len(x), reverse=True)

In [26]:
with open('output.json', 'w') as f:
    json.dump(clusters_texts, f)

In [27]:
len(clusters_texts)

2534

# посмотрим на названия, которые не попали в кластеры по порогу

In [28]:
final_top_full[(final_top_full['CANDIDATE_ID'].isin(np.array(no_clusters).squeeze()))
              &(final_top_full.RANK<=2)
              ][['CANDIDATE_NAME', 'ETALON_NAME']].head(100)

,CANDIDATE_NAME,ETALON_NAME
34912,Шампанское AOC Champagne Moet&amp Chandon Nect...,Шампанское AOC Champagne Moet&amp Chandon Brut...
13004,"Водка Пять озер Премиум 0,5л Россия","Водка ПЯТЬ ОЗЕР Премиум, 0,5л"
13014,"Водка ПЯТЬ ОЗЕР Премиум, 0,5л","Водка Пять озер Премиум 0,5л Россия"
19004,"Коньяк Remy Martin XO в п/у 0,35л Франция","Коньяк REMY MARTIN XO, 0,35л"
19024,"Коньяк REMY MARTIN XO, 0,35л","Коньяк Remy Martin XO в п/у 0,35л Франция"
...,...,...
11114,"Газированный напиток COCA-COLA, 2л",Напиток Coca-Cola 2л
24087,"Пивной напиток Corona Extra светлое ст/б 0,355...",Пивной напиток светлое CORONA Extra лагер стек...
24077,Пивной напиток светлое CORONA Extra лагер стек...,"Пивной напиток Corona Extra светлое ст/б 0,355..."
4391,Перчатки Paclan латексные размер М 10 шт,Перчатки Paclan латексные М 10шт Китай


# Вывод

Алгоритм разработан для полу-автоматического поиска дубликатов, на кластеры смотреть особого смысла нет, т.к. есть дополнительные штрафы на несовпадение цифр(граммовок/объемов) и т.п.

лучше смотреть на топ совпадений для каждого кандидата, таким образом и предполагается использование этого инструмента: человек просматривает на топ для каждого нового наименования и выбирает наиболее подходящее, либо добавляет его в список эталонов

Алгоритм имеет возможность работать по батчам и параллельно, для лучшей масштабируемости

## возможные улучшения

необходимо заводить производителей и страну производителя как отдельный признак, и при поступлении новых наименований заполнять соответствующим образом базу (заполняя необходимые поля)